## Compute execution ratio using dedicated service(C#)

### Overview
Tick count enriched with trade count can be used as an indicator to illustrate market activity and measure market quality by:
* computing the technical transactions/transmissions count on a tick by tick basis, *tick count*,
* comparing it to the total *trade count*

Correlated with a set of other indicators such as traded volume and price volatility, over the same look back period and the same sampling granularity, it repserents a good measurement of the market quality.

This samples enables to retrieve **on-demand** tick count data points (L1 and L1L2) by calling a dedicated service.

### Inputs/outputs
Execution ratio sample requires instrument's identifier, date time intervals and time granularity as per inputs. It returns the tick count Vs trade count sampled using the input time granularity.

This sample also shows how to plot a simple graph for basis financial analysis using an open source library.

<div class="alert alert-block alert-info">
    <b>Note:</b> Similar scenarios are available and enable to enrich tick count with price volatility or trade sizes.
</div>

### Services used
This sample uses *gRPC requests* in order to retrieve tick by tick data from the dedicated hosted service. The queried endpoint in this script are:
* *TopologiesService*: to directly retrieve tick by tick data data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Topology.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run tick count sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Topology.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress(System.Environment.GetEnvironmentVariable("GRPC_APIS"));

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *TopologiesService*, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Topology request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Execution ratio granularity

In [ ]:
var granularity = TopologyGranularity.Daily;

#### 3.2 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate Trades request
var requestL1 = new TopologiesRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Granularity = granularity,
    Level = Level.Trades
};

// Generate Trades and Book request
var requestL2 = new TopologiesRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker}, 
    Granularity = granularity,
    Level = Level.TradesAndBook
};

In [ ]:
// Instantiate the topologies service
var service = new TopologiesService.TopologiesServiceClient(channel);        

#### 3.3 Retrieve data

In [ ]:
// Process Trades request
var replyL1 = service.Topologies(requestL1, headers);
 
// Retrieve topologies service response for Trades
var topologiesL1 = replyL1.Entries;
var totalTicksL1 = topologiesL1.Sum(t=> (long)t.TicksCount);

display(topologiesL1)
// Store tick count starting from the date of your choice
//var tickCountsL1 = topologiesL1.Where(t => t.Begin.Year >= 2020).Select(t => new Tuple<DateTime, ulong>(new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount)).ToArray();

In [ ]:
// Process Trades and Book request
var replyL2 = service.Topologies(requestL2, headers);
 
// Retrieve topologies service response for Trades and Book
var topologiesL2 = replyL2.Entries;
var totalTicksL2 = topologiesL2.Sum(t=> (long)t.TicksCount);

// Store tick count starting from the date of your choice
// var tickCountsL2 = topologiesL2.Where(t => t.Begin.Year >= 2020).Select(t => new Tuple<DateTime, ulong>(new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount)).ToArray();

### Step4: Visualize data

#### 4.1 Retrieve tick counts
The following code snippet retrieves *Trades ticks* and *Trades and Book ticks* counts:

In [ ]:
// Extract ticks count for Trades
var trades = topologiesL1.Select(t => new Tuple<DateTime, double>(new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount));
// Extract the price Trades and Book
var tradesAndBook = topologiesL2.Select(t => new Tuple<DateTime, double>(new DateTime(t.Begin.Year, t.Begin.Month, t.Begin.Day), t.TicksCount));

#### 4.2 Compute execution ratio
The following code snippet computes execution ratio by dividing  *Trades ticks* and *Trades and Book ticks* counts:

In [ ]:
var executionRatio = new Dictionary<DateTime, double>();
var i = 0;
foreach (var elt in trades) {
    executionRatio[elt.Item1] = 100 * elt.Item2 / topologiesL2[i].TicksCount;
    i += 1;
}

//display(executionRatio)

#### 4.2 Plot execution ratio

In [ ]:
var executionRatiochart = new List<Scattergl>
{
    new Scattergl
    {
        name = "Trades ticks count",
        mode = "lines",
        x = trades.Select(b => b.Item1),
        y = trades.Select(b => b.Item2)
    },
    new Scattergl
    {
        name = "Trades and Book ticks count",
        mode = "lines",
        x = tradesAndBook.Select(b => b.Item1),
        y = tradesAndBook.Select(b => b.Item2)
    },
    new Scattergl
    {
        name = "Execution ratio",
        mode = "lines",
        x = executionRatio.Keys,
        y = executionRatio.Values,
        yaxis = "y2"
    }
};


var layout = new Layout.Layout{ 
    title = $"Execution ratio for {ticker}",
    yaxis = new Yaxis {
        title = "Ticks count", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Ratio", 
        autorange = true,
        side = "right",  
    },
};

var chart = Chart.Plot(executionRatiochart, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);